In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import warnings
import seaborn as sns


In [2]:
df=pd.read_csv(r'crop_production.csv')

In [10]:
# Droping Nan Values
data = df.dropna()
print(data.shape)
test = df[~df["Production"].notna()].drop("Production",axis=1)
print(test.shape)

(242361, 7)
(3730, 6)


In [24]:
sum_maxp = data["Production"].sum()
data["percent_of_production"] = data["Production"].map(lambda x:(x/sum_maxp)*100)

C:\Users\ankit\AppData\Local\Temp\ipykernel_17508\1048416122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["percent_of_production"] = data["Production"].map(lambda x:(x/sum_maxp)*100)


In [28]:
data1 = data.drop(["District_Name","Crop_Year"],axis=1)

In [58]:
data1

,State_Name,Season,Crop,Area,Production,percent_of_production
0,Andaman and Nicobar Islands,Kharif,Arecanut,1254.0,2000.0,1.416670e-06
1,Andaman and Nicobar Islands,Kharif,Other Kharif pulses,2.0,1.0,7.083351e-10
2,Andaman and Nicobar Islands,Kharif,Rice,102.0,321.0,2.273756e-07
3,Andaman and Nicobar Islands,Whole Year,Banana,176.0,641.0,4.540428e-07
4,Andaman and Nicobar Islands,Whole Year,Cashewnut,720.0,165.0,1.168753e-07
...,...,...,...,...,...,...
246086,West Bengal,Summer,Rice,306.0,801.0,5.673764e-07
246087,West Bengal,Summer,Sesamum,627.0,463.0,3.279592e-07
246088,West Bengal,Whole Year,Sugarcane,324.0,16250.0,1.151045e-05
246089,West Bengal,Winter,Rice,279151.0,597899.0,4.235129e-04


In [ ]:
data_dum = pd.get_dummies(data1)
data_dum[:5]

,Area,Production,percent_of_production,State_Name_Andaman and Nicobar Islands,State_Name_Andhra Pradesh,State_Name_Arunachal Pradesh,State_Name_Assam,State_Name_Bihar,State_Name_Chandigarh,State_Name_Chhattisgarh,...,Crop_Turmeric,Crop_Turnip,Crop_Urad,Crop_Varagu,Crop_Water Melon,Crop_Wheat,Crop_Yam,Crop_other fibres,Crop_other misc. pulses,Crop_other oilseeds
0,1254.0,2000.0,1.416670e-06,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2.0,1.0,7.083351e-10,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,102.0,321.0,2.273756e-07,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,176.0,641.0,4.540428e-07,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,720.0,165.0,1.168753e-07,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [57]:
ohe_columns = data_dum.drop("Production", axis=1).columns.tolist()

# Save to a pickle file
with open("ohe_columns.pkl", "wb") as f:
    pickle.dump(ohe_columns, f)

In [34]:
x = data_dum.drop("Production",axis=1)
y = data_dum[["Production"]]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30, random_state=42)
print("x_train :",x_train.shape)
print("x_test :",x_test.shape)
print("y_train :",y_train.shape)
print("y_test :",y_test.shape)

x_train : (169652, 165)
x_test : (72709, 165)
y_train : (169652, 1)
y_test : (72709, 1)


In [35]:
x_train[:5]

,Area,percent_of_production,State_Name_Andaman and Nicobar Islands,State_Name_Andhra Pradesh,State_Name_Arunachal Pradesh,State_Name_Assam,State_Name_Bihar,State_Name_Chandigarh,State_Name_Chhattisgarh,State_Name_Dadra and Nagar Haveli,...,Crop_Turmeric,Crop_Turnip,Crop_Urad,Crop_Varagu,Crop_Water Melon,Crop_Wheat,Crop_Yam,Crop_other fibres,Crop_other misc. pulses,Crop_other oilseeds
227784,674.0,4.349178e-07,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
73466,36.0,2.620840e-08,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
148845,361.0,1.246670e-07,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
209150,24.0,1.983338e-08,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
30541,330.0,5.395389e-06,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(x_train,y_train)
preds = model.predict(x_test)

c:\Users\ankit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [37]:
from sklearn.metrics import r2_score
r = r2_score(y_test,preds)
print("R2score when we predict using Randomn forest is ",r)

R2score when we predict using Randomn forest is  0.9979901934937807


In [ ]:
'''from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train)'''

In [51]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



print("Mean Squared Error:", round(mse, 4))
print("Mean Absolute Error:", round(mae, 4))
print("R-squared:", round(r2, 4))


Mean Squared Error: 691388924863.443
Mean Absolute Error: 7481.2138
R-squared: 0.998


In [55]:
import pickle
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np



# ✅ Regression Metrics
r2 = r2_score(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
mae = mean_absolute_error(y_test, preds)

print("R² Score:", round(r2, 4))
print("RMSE:", round(rmse, 4))
print("MAE:", round(mae, 4))

# ✅ Save Pickle files
with open("rf_model.pkl", "wb") as f:
    pickle.dump(model, f)



print("✅ Random Forest model, scaler, and encoders saved.")


R² Score: 0.998
RMSE: 831498.0005
MAE: 7481.2138
✅ Random Forest model, scaler, and encoders saved.
